# P4DAMP Experiments

## Objective

To evaluate P4DAMP accuracy under different mitigation thresholds. 

## Parameters


Parameter | Value | Description
----------| ----- | -----------
a		  | 0.2	  | Attack Proportion	
k		  | 4.125 | Sensitivity Coefficient
w		  | 1280  | Count-Sketch Width
m		  | 2^13  | Window Size
n         | 69    | Total Windows

## Initial Traffic Characterization

Observation Window Number: 188

Values obtained from line 188 of [ddos5z8-4.125.tcad.txt](file:///d:/Downloads/P4/logs-ddos5xyz/ddos5z8-4.125.tcad.txt).

This log file was generated as described in experiment 5Z8.

Parameter | Value 
----------| ----- 
src_ewma  |	2500001
src_ewmmd |	38953
dst_ewma  |	2452881
dst_ewmmd |	36773  



## Factors

Factor | Levels | Description
------ | ------ | -----------
t      | {80;40;20;10} |  Mitigation thresholds

## Metrics

Proportions of correctly- and incorrectly-classified packets.

How to calculate these metrics?

- Initialize the switch.
    - Note: we can reset it by zeroing the registers.
- Start capturing the outgoing packets.
    - veth3: classified as legitimate
    - veth5: classified as malicious 
- Play the workload traffic.
- Obtain the following numbers of packets:
    - Legitimate PCAP:    [saved as p4sb3e01n.pcap.gz]
        - Marked      incorrectly classified          
        - Unmarked    correctly classified
    - Malicious PCAP:     [saved as p4sb3e01p.pcap.gz]
        - Marked      correctly classified
        - Unmarked    incorrectly classified

**Note: Deduct 5x8192=40960 packets from the legitimate count (first 5 OWs).**

## Dataset

Files saved [here](file:///d:/Downloads/P4/ddos/datasets/zed)

### zed20percent-fast.pcap 

Dataset has 192 OWs of 2^13 packets each.

We utilize OWs 1 to 69 [69 * 8192 = 565248 packets]

        tcpreplay --limit 565248
        
Wireshark display filters:

        Good packets: frame[50:1]==0 && frame.number<=565248
        Bad packets: frame[50:1]==1 && frame.number<=565248

Generation:

This dataset skips the training and pre-attack parts of zed20percent.pcap.
These correspond to the first 188 OWs. So:

- First packet:	(189-1) * 8192 + 1 = 1540097 
- Last packet:	384 * 8192 = 3145728

        editcap -r .\zed20percent.pcap .\zed20percent-fast.pcap 1540097-3145728

Attack starts at t=5. 

We ignore measurements from t=1 to t=5. 


### zed20percent.pcap 

Dataset has 384 OWs of 2^13 packets each. 

Sources:

- ddos5_00000_20160406100300.pcap - Volume 00 (known to contain only legitimate packets)
- ddostrace.20070804_141436.pcap - Generated directly from CAIDA DDoS Attack Traces, as follows:

        mergecap -w ddostrace.20070804_141436.pcap 
            .\ddostrace.to-victim.20070804_141436.pcap 
            .\ddostrace.from-victim.20070804_141436.pcap

        trafg -n 2097152 -a 0.2 
            ddos5_00000_20160406100300.pcap 
            ddostrace.20070804_141436.pcap 
            zed20percent.pcap

## Observations

We seem to have reached the maximum performance achievable under this observation window size.

## Next steps

Increase the size of the workload to 2^24 packets. 

Perform a factorial experiment using:

Factor | Levels | Description
------ | ------ | -----------
m | 2^i, for i = 13..18 | Observation Window Length
t | 0.001\*j\*m, for j =1..20 | Mitigation Threshold

To do this experiment:

- Generate the base workload.
- For each m:
    - Process the workload with EE and save the log. 
    - Use an experimental script to obtain TCAD readings for several k coefficients. 
    - Analyze the TCAD readings to find the optimal k coefficient. 
    - Analyze the TCAD readings to find the traffic characterization values under the optimal k coefficient. 
    - Use an experimental script to obtain the classification results related to different levels of t.
        - (Re)initialize the registers on the data plane. 
        - Replay the workload. 
        - Collect pcaps.
        - Collect statistics. 

## Synthetic Workload

**Calculating trafg parameters**

    Trafg takes a packet_count and generates:                               Volume     
        Training phase:                                                         
            packet_count/2 legitimate traffic                               1/3
        Detection phase:
            packet_count/4 legitimate traffic                               1/6
            packet_count/2 legitimate + malicious traffic:                  1/3
                malicious:         attack_proportion  * packet_count/2      
                legitimate:     (1-attack_proportion) * packet_count/2  
            packet_count/4 legitimate traffic                               1/6

In [1]:
import math

detection_phase_packets = 2**24         # Phases Two, Three, and Four. 

trafg_packets = detection_phase_packets
total_packets = int(1.5 * trafg_packets)
attack_proportion = 0.2

pcap_legitimate = '"/media/p4/p4damp/datasets/caida/caida-vol-00-to-vol-05.pcap"'
pcap_attack = '"/media/p4/p4damp/datasets/caida/ddostrace.20070804_141436.pcap"'
pcap_out = '"/media/p4/p4damp/datasets/ddos20/ddos20.pcap"'

needed_attack_packets = math.ceil(0.5 * detection_phase_packets * attack_proportion)
needed_legitimate_packets = math.ceil(1.5 * trafg_packets - needed_attack_packets)

print("Detection phase packets:", f"{detection_phase_packets:,}")
print("Trafg packets:", f"{trafg_packets:,}")
print("Total packets:", f"{total_packets:,}")
print("")
print("Needed legitimate packets:", f"{needed_legitimate_packets:,}")
print("Needed attack packets:", f"{needed_attack_packets:,}")
print("")
print("Command line:","/media/p4/ddosd-cpp/bin/trafg -n", 
      trafg_packets, "-a", attack_proportion, pcap_legitimate, pcap_attack, pcap_out)

Detection phase packets: 16,777,216
Trafg packets: 16,777,216
Total packets: 25,165,824

Needed legitimate packets: 23,488,102
Needed attack packets: 1,677,722

Command line: /media/p4/ddosd-cpp/bin/trafg -n 16777216 -a 0.2 "/media/p4/p4damp/datasets/caida/caida-vol-00-to-vol-05.pcap" "/media/p4/p4damp/datasets/caida/ddostrace.20070804_141436.pcap" "/media/p4/p4damp/datasets/ddos20/ddos20.pcap"


Note: Lapolli's synthetic workload uses the following files.

- Legitimate (~120 Mpackets): 
    - equinix-chicago.dirA.20160406-130300.UTC.anon.pcap 
    - equinix-chicago.dirA.20160406-130400.UTC.anon.pcap
    - equinix-chicago.dirA.20160406-130500.UTC.anon.pcap
    - equinix-chicago.dirA.20160406-130600.UTC.anon.pcap
- Malicious (~25 Mpackets): 
    - ddostrace.to-victim.20070804_141436.pcap 
    - ddostrace.from-victim.20070804_141436.pcap
    
I can obtain a ~45 Mpacket legitimate PCAP by concatenating the first **six** volumes. 

I'm concatenating these six volumes into caida-vol-00-to-vol-05.pcap:

    mergecap -a -w "D:\Downloads\P4\ddos\datasets\p4damp\caida-vol-00-to-vol-05.pcap" \
    "D:\Downloads\P4\ddos\datasets\aclapolli\synthetic\ddos5\ddos5_00000_20160406100300.pcap" \
    "D:\Downloads\P4\ddos\datasets\aclapolli\synthetic\ddos5\ddos5_00001_20160406100307.pcap" \
    "D:\Downloads\P4\ddos\datasets\aclapolli\synthetic\ddos5\ddos5_00002_20160406100315.pcap" \
    "D:\Downloads\P4\ddos\datasets\aclapolli\synthetic\ddos5\ddos5_00003_20160406100322.pcap" \
    "D:\Downloads\P4\ddos\datasets\aclapolli\synthetic\ddos5\ddos5_00004_20160406100329.pcap" \
    "D:\Downloads\P4\ddos\datasets\aclapolli\synthetic\ddos5\ddos5_00005_20160406100337.pcap"
    
**I'm now checking the packet count in the legitimate PCAP.** 

    > capinfos "D:\Downloads\P4\ddos\datasets\p4damp\caida-vol-00-to-vol-05.pcap"
    
This results in: 

    File name:           .\caida-vol-00-to-vol-05.pcap
    File type:           Wireshark/... - pcapng
    File encapsulation:  Ethernet
    File timestamp precision:  microseconds (6)
    Packet size limit:   file hdr: (not set)
    Number of packets:   49 M
    File size:           4128 MB
    Data size:           2555 MB
    Capture duration:    45,111394 seconds
    First packet time:   2016-04-06 10:03:00,000000
    Last packet time:    2016-04-06 10:03:45,111394
    Data byte rate:      56 MBps
    Data bit rate:       453 Mbps
    Average packet size: 52,00 bytes
    Average packet rate: 1089 kpackets/s
    SHA256:              bde16c362f2c2df7febef49f136d2fcc7cd9b1c5344a9b84e912a24995456260
    RIPEMD160:           87ec283917838c957218aad8b656d47f152f9310
    SHA1:                a4fc4e27b0f1378862823e5c9c846adeedb8486f
    Strict time order:   True
    Capture oper-sys:    64-bit Windows 10 (1803), build 17134
    Capture application: Mergecap (Wireshark) 3.0.3 (v3.0.3-0-g6130b92b0ec6)
    Number of interfaces in file: 1
    Interface #0 info:
                         Encapsulation = Ethernet (1 - ether)
                         Capture length = 1500
                         Time precision = microseconds (6)
                         Time ticks per second = 1000000
                         Number of stat entries = 0
                         Number of packets = 49152000

These stats indicate that I have 49 Mpackets of legitimate traffic. 

**I'm now checking the stats of the malicious PCAP.**

    > capinfos "D:\Downloads\P4\ddos\datasets\caida\2007 ddostrace\ddostrace.20070804_141436.pcap"
    
This results in: 
        
    File name:           D:\Downloads\P4\ddos\datasets\caida\2007 ddostrace\ddostrace.20070804_141436.pcap
    File type:           Wireshark/... - pcapng
    File encapsulation:  Raw IP
    File timestamp precision:  microseconds (6)
    Packet size limit:   file hdr: (not set)
    Packet size limit:   inferred: 20 bytes - 52 bytes (range)
    Number of packets:   26 M
    File size:           1650 MB
    Data size:           1622 MB
    Capture duration:    299,999612 seconds
    First packet time:   2007-08-04 18:14:36,485318
    Last packet time:    2007-08-04 18:19:36,484930
    Data byte rate:      5407 kBps
    Data bit rate:       43 Mbps
    Average packet size: 60,63 bytes
    Average packet rate: 89 kpackets/s
    SHA256:              00b97864a41c2d310949eabfcc023f96504cd3d6321f05b3de7effd19e2920b1
    RIPEMD160:           1b7134879ea32a207833dcc5be97e9dbab5dc8da
    SHA1:                a63bdb6648a9d9ab841f851179f2fcc485a0f1d9
    Strict time order:   True
    Capture oper-sys:    64-bit Windows 10 (1803), build 17134
    Capture application: Mergecap (Wireshark) 3.0.2 (v3.0.2-0-g621ed351d5c9)
    Number of interfaces in file: 1
    Interface #0 info:
                         Encapsulation = Raw IP (7 - rawip)
                         Capture length = 65536
                         Time precision = microseconds (6)
                         Time ticks per second = 1000000
                         Number of stat entries = 0
                         Number of packets = 26760675

These stats indicate that I have 26 Mpackets of malicious traffic.

**I'm now running this trafg command line to generate the workload:**

    /media/p4/ddosd-cpp/bin/trafg -n 16777216 -a 0.2 \
    "/media/p4/p4damp/datasets/caida/caida-vol-00-to-vol-05.pcap" \
    "/media/p4/p4damp/datasets/caida/ddostrace.20070804_141436.pcap" \
    "/media/p4/p4damp/datasets/ddos20/ddos20.pcap"

Timestamps output by trafg:

    Attack start: 1459947791531438
    Attack end:   1459947797619262 

In [4]:
from datetime import datetime, timedelta

epoch = datetime(1970,1,1)

attack_start_usec = 1459947791531438
attack_end_usec = 1459947797619262

attack_start = epoch + timedelta(microseconds=attack_start_usec)
attack_end = epoch + timedelta(microseconds=attack_end_usec)

print("Attack start:", attack_start)
print("Attack end:", attack_end)

Attack start: 2016-04-06 13:03:11.531438
Attack end: 2016-04-06 13:03:17.619262


**I'm now checking the stats of the PCAP I have just generated.**

    > capinfos D:\Downloads\P4\p4damp\datasets\ddos20\ddos20.pcap
    
This results in:

    File name:           D:\Downloads\P4\p4damp\datasets\ddos20\ddos20.pcap
    File type:           Wireshark/tcpdump/... - pcap
    File encapsulation:  Ethernet
    File timestamp precision:  microseconds (6)
    Packet size limit:   file hdr: 1500 bytes
    Number of packets:   25 M
    File size:           1711 MB
    Data size:           1308 MB
    Capture duration:    21.415469 seconds
    First packet time:   2016-04-06 10:03:00.000000
    Last packet time:    2016-04-06 10:03:21.415469
    Data byte rate:      61 MBps
    Data bit rate:       488 Mbps
    Average packet size: 52.00 bytes
    Average packet rate: 1175 kpackets/s
    SHA256:              f4ae1d60c310dd98efc35ae491b5e9de3f5f1c1caaf7bbe6ff2cdbd8a65ea408
    RIPEMD160:           3a927d0e591960a153ffb4786d73183cf6ab0bbf
    SHA1:                d27d604cd9f50814dfe50225e2c22ed26f772cf2
    Strict time order:   True
    Number of interfaces in file: 1
    Interface #0 info:
                         Encapsulation = Ethernet (1 - ether)
                         Capture length = 1500
                         Time precision = microseconds (6)
                         Time ticks per second = 1000000
                         Number of stat entries = 0
                         Number of packets = 25165824                    

I'm now calculating the offsets of each phase start and end.

In [8]:
n = 25165824 # Number of packets generated by trafg, contained in ddos20.pcap.

phases_0_start = 1 
phases_0_end = phases_0_start + n/3 - 1
phases_1_start = phases_0_end + 1
phases_1_end = phases_1_start + n/6 - 1
phases_2_start = phases_1_end + 1
phases_2_end = phases_2_start + n/3 - 1
phases_3_start = phases_2_end + 1
phases_3_end = phases_3_start + n/6 - 1

print("Training:",int(phases_0_start),"-",int(phases_0_end))
print(" Phase 1:",int(phases_1_start),"-",int(phases_1_end))
print(" Phase 2:",int(phases_2_start),"-",int(phases_2_end))
print(" Phase 3:",int(phases_3_start),"-",int(phases_3_end))

Training: 1 - 8388608
 Phase 1: 8388609 - 12582912
 Phase 2: 12582913 - 20971520
 Phase 3: 20971521 - 25165824


## Generating Entropy Estimation Traces

I must run EE for each level of the window size factor (m): 2^i, for i = 13..18. 

I have created the following JSON files, one for each level of m:

    ee_m_2_13.json
    ee_m_2_14.json
    ee_m_2_15.json
    ee_m_2_16.json
    ee_m_2_17.json
    ee_m_2_18.json
    
These files are under D:\Downloads\P4\ddosd-p4\scripts.    

I've written a script to automatically invoke **ee** and save a series of entropy estimation traces.


In [9]:
# /media/p4/ddosd-p4/scripts/p4d_ddos20/ee_m_levels.py

import os

ee_bin = "/media/p4/ddosd-cpp/bin/ee"
json_path = "/media/p4/ddosd-p4/scripts/p4d_ddos20"
log_path = "/media/p4/p4damp/datasets/ddos20/ddos20_results"
workload_file = "/media/p4/p4damp/datasets/ddos20/ddos20.pcap" 

for i in range(13,19):
    base_name = "ee_m_2_" + str(i) 
    json_file = json_path + "/" + base_name + ".json"
    log_file = log_path + "/" + base_name + ".log"
    ee_cmd = ee_bin + " -c " + json_file + " " + workload_file + " > " + log_file 
#     print(ee_cmd)
#     os.system(ee_cmd)


After running the script, the following logs were saved under /media/p4/p4damp/datasets/ddos20/ddos20_results:

    ee_m_2_13.log
    ee_m_2_14.log
    ee_m_2_15.log
    ee_m_2_16.log
    ee_m_2_17.log
    ee_m_2_18.log

## Choosing a Sensitivity Coefficient

The choice of a sensitivity coefficient must be criterious. Lapolli originally observed that lower values of k tend to keep the true positive rate near 100%, which suggests it would be wise to use the lowest values of k that still give us a 'low enough', i.e., less than 2% false positive rate. 

Finding the optimal value of k can be done through these macro steps:

Obtaining statistics:

- For m from 13 to 18:
    - For k from 0 to 5 by 0.125: 
        - Run the **ee** trace file through **tcad**, using **k** as a parameter. 
        - Save the **tcad** trace file. 
        - Calculate statistics (TPR, FPR, etc.) and save them to a summary file. 

Analyzing statistics:

- For m from 13 to 18:
    - Analyze the summary file to find the lowest value of k which yields FPR <= 2.  

I've written a script to automatically invoke **tcad** and save its output: 

    D:\Downloads\P4\ddosd-p4\scripts\p4d_ddos20\tcad_m_levels.py

Our experiments with **tcad_m_levels.py** show us candidate values for k, as follows: 

Log2(m) | Sensitivity Coefficient | FPR
--------|-------------------------|----
13      | 4.000 | 1.8%
14      | 4.125 | 1.7%
15      | 4.250 | 1.8%
16      | 4.500 | 1.6%
17      | 4.750 | 1.6%
18      | 3.625 | 0.0%

Furthermore, analysis of the results shows that it is not worth it to perform tests with log2(m) = 13, 15, and 17. Therefore, we will restric our efforts to the most interesting values, i.e., log2(m) = 14, 16, and 18. 

## Obtaining Traffic Characterization Values

This is important to avoid wasting processor time.

**The total packet number used as an input for trafg is n=2^24=16,777,216**

The number of training phase packets is tp = n/2 = 2^24/2 =2^23. 

The number of observation windows is t = tp/m = 2^(log2(tp)-log2(m)) = 2^(tp-m) = 2^(23-m) 

In [15]:
for m in range(14,19,1):
    t_exp = 23-m
    t_end_ow = 2**t_exp
    print("tp = 2^23" + ", " + 
          "m = 2^" + str(m) + ", " + 
          "t = 2^" + str(t_exp) + ", " +
          "t_end_ow = " + str(t_end_ow))

ph1_start_packet = 2**23 + 1
print("ph1_start_packet = " + str(ph1_start_packet))  
ph2_start_packet = 2**23 + 2**22 + 1
print("ph2_start_packet = " + str(ph2_start_packet))  


tp = 2^23, m = 2^14, t = 2^9, t_end_ow = 512
tp = 2^23, m = 2^15, t = 2^8, t_end_ow = 256
tp = 2^23, m = 2^16, t = 2^7, t_end_ow = 128
tp = 2^23, m = 2^17, t = 2^6, t_end_ow = 64
tp = 2^23, m = 2^18, t = 2^5, t_end_ow = 32
ph1_start_packet = 8388609
ph2_start_packet = 12582913


### Finding TC Values 

We need to process **tcad** logs under an experimental script.

I have written a script to obtain the training values for specific k-values. 

After running my tcad_m_training_values.py script, I've obtained:

    p4@p4:/media/p4/ddosd-p4/scripts/p4d_ddos20$ ./tcad_m_training_values.py 
    
    Trace file: tcad_m_2_14_k_4.125.log
    Last training OW: 1024
    register_write ingress.log2_m 0 14
    register_write ingress.training_len 0 0
    register_write ingress.alpha 0 20
    register_write ingress.k 0 33
    register_write src_ewma 0 2628797
    register_write src_ewmmd 0 18220
    register_write dst_ewma 0 2572820
    register_write dst_ewmmd 0 20993
    register_write migration_t 0 10

    Trace file: tcad_m_2_16_k_4.500.log
    Last training OW: 256
    register_write ingress.log2_m 0 16
    register_write ingress.training_len 0 0
    register_write ingress.alpha 0 20
    register_write ingress.k 0 36
    register_write src_ewma 0 2869138
    register_write src_ewmmd 0 13911
    register_write dst_ewma 0 2811871
    register_write dst_ewmmd 0 14734
    register_write migration_t 0 10

    Trace file: tcad_m_2_18_k_3.625.log
    Last training OW: 64
    register_write ingress.log2_m 0 18
    register_write ingress.training_len 0 0
    register_write ingress.alpha 0 20
    register_write ingress.k 0 29
    register_write src_ewma 0 3053057
    register_write src_ewmmd 0 10670
    register_write dst_ewma 0 2991188
    register_write dst_ewmmd 0 15945
    register_write migration_t 0 10

These values indicate the parameters I should:

- Observe when running the pipeline (for validation)
- Use to initialize the pipeline while skipping training

I saved these instructions in these files: 

    D:\Downloads\P4\ddosd-p4\scripts\p4d_ddos20\control_rules_m_2_14.txt
    D:\Downloads\P4\ddosd-p4\scripts\p4d_ddos20\control_rules_m_2_16.txt
    D:\Downloads\P4\ddosd-p4\scripts\p4d_ddos20\control_rules_m_2_18.txt
    
I saved the basic pipeline configuration instructions in this file: 

    D:\Downloads\P4\ddosd-p4\scripts\p4d_ddos20\control_rules_base.txt

I'm now investigating how to reset the registers.

Found the simple_switch_cli instruction **reset_state**.

### New Lab Environment 

I edited the Makefile and a few more things in the environment.

My first experiment in the new environment:

    make 
    make veth_start
    make run_without_config
    make exp_zed_10
    
The **make exp_zed_10** command launches the following part of the Makefile:

    SS_PREFIX="/home/p4/p4sec/aclapolli-bmv2/targets/simple_switch"
    SS_CLI=$(SS_PREFIX)/simple_switch_CLI
    TCPREPLAY=sudo tcpreplay --preload-pcap --quiet

    PACKET_LIMIT=565248
    PACKET_RATE=3072
    PCAP_FILE=/media/p4/p4damp/datasets/zed/zed20percent-fast.pcap

    exp_zed_10:
        $(SS_CLI) < /media/p4/ddosd-p4/scripts/p4d_ddos20/control_rules_base.txt
        $(SS_CLI) < /media/p4/ddosd-p4/scripts/p4d_ddos20/control_rules_zed.txt
        echo "register_write mitigation_t 0 10" | $(SS_CLI) 
        ./$(SCRIPT_DIR)/run_capture_to_files.sh start $(OUTPUT_DIR)
        $(TCPREPLAY) --limit=$(PACKET_LIMIT) --pps=$(PACKET_RATE) --pps-multi=16 -i veth0 $(PCAP_FILE) 2>&1
        ./$(SCRIPT_DIR)/run_capture_to_files.sh stop $(OUTPUT_DIR)

       
Tcpreplay then runs the pcap through the switch. 

At last, we've obtained the same results as we had when running everything by hand.

This is excellent because it means that our experimental environment is healthy! :D 


        

Generating "notraining" PCAP:

    D:\Downloads\P4\p4damp\datasets\ddos20>editcap -r ddos20.pcap ddos20-notraining.pcap 8388609-25165824 
    
Getting file information: 

    D:\Downloads\P4\p4damp\datasets\ddos20>capinfos ddos20-notraining.pcap
    File name:           ddos20-notraining.pcap
    File type:           Wireshark/... - pcapng
    File encapsulation:  Ethernet
    File timestamp precision:  microseconds (6)
    Packet size limit:   file hdr: (not set)
    Number of packets:   16 M
    File size:           1409 MB
    Data size:           872 MB
    Capture duration:    13.708319 seconds
    First packet time:   2016-04-06 10:03:07.707150
    Last packet time:    2016-04-06 10:03:21.415469
    Data byte rate:      63 MBps
    Data bit rate:       509 Mbps
    Average packet size: 52.00 bytes
    Average packet rate: 1223 kpackets/s
    SHA256:              2cd3c289dec9f3f72a7a177e4e3b3365e01c41ae815bc5fa5608dc85e71bd266
    RIPEMD160:           e52dd6466608a9137d3925c1d7708527c6979bb2
    SHA1:                6f06bdd1c29421726c036112302ea681e3116bf8
    Strict time order:   True
    Capture application: Editcap (Wireshark) 3.0.3 (v3.0.3-0-g6130b92b0ec6)
    Number of interfaces in file: 1
    Interface #0 info:
                         Encapsulation = Ethernet (1 - ether)
                         Capture length = 1500
                         Time precision = microseconds (6)
                         Time ticks per second = 1000000
                         Number of stat entries = 0
                         Number of packets = 16777216
                         
**Our PACKET_LIMIT should be 16777216.**                   

In [27]:
run_packets = 2**24
packet_rate = 3072 # pps
run_time_seconds = int(run_packets/packet_rate)

print("Estimated time for each run:", run_time_seconds, "seconds.")

total_run_time_hours = int(20 * 3 * run_time / 3600) # We have 20 levels of t, and 3 levels of m. 

print("Estimated time for all runs:", total_run_time_hours, "hours.")


Estimated time for each run: 5461 seconds.
Estimated time for all runs: 91 hours.


In [5]:
from datetime import datetime, timedelta

epoch = datetime(1970,1,1)

ts_usec = 1459947839999998

ts_date = epoch + timedelta(microseconds=ts_usec)

print("Timestamp:", ts_date)


Timestamp: 2016-04-06 13:03:59.999998
